# Deep Learning Notebook of project Building-Perceptron

## Introduction (must be done in english)

## Imports

In [3]:
# !pip install --upgrade pip
# !pip install pandas

In [4]:
import pandas as pd
import numpy as np

## Data Exploration

### Loading Data

In [10]:
original_df = pd.read_csv("data/bcw_data.csv")
df = original_df.copy()

### Exploratory analysis

In [11]:
df.head(2)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


### Data Cleaning

## Preparing for Deep Learning

### Feature Selection / Dimensionality Reduction

#### Boruta

### Preprocessing

### Choosing Metrics

In [17]:
# On va justifier le choix des différentes métriques qu'on va utiliser pour le projet (c'est spécifié plusieurs)

## Modelisation

### Importing the ClassPerceptron .py script

In [15]:
# from **** import Laclass

### Comparing with a very basic Deep Learning model

In [18]:
# Sigmoid machin machin relou

## Evaluating the model results

## Conclusion

``Concluez sur l’efficacité d’un Perceptron sur ce type de problématique
puis, proposez des solutions sans les implémenter.``

Je conclute tu conclute

petit passage comme quoi notre perceptron est nul et c'est en explorant d'autres stratégies comme le backtracking qu'on attendra de très bon résultat